In [114]:
import numpy as np
import pandas as pd


from sklearn.preprocessing import power_transform
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [115]:
# reading two csv files
data1 = pd.read_csv('data/train.csv',dtype={'Store':'string', 'StateHoliday':'string'})
data2 = pd.read_csv('data/store.csv',dtype={'Store':'string', 'StoreType':'string', 'Assortment':'string'})
  
# using merge function by setting how='inner'
total = pd.merge(data1, data2, 
                   on='Store', 
                   how='inner')

In [116]:
total

,Date,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,2013-01-01,1115,2.0,0.0,0.0,0.0,0.0,a,1.0,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec"
1,2013-01-02,1115,3.0,3697.0,305.0,1.0,0.0,0,1.0,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec"
2,2013-01-03,1115,4.0,4297.0,300.0,1.0,0.0,0,1.0,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec"
3,2013-01-04,1115,5.0,4540.0,326.0,1.0,0.0,0,1.0,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec"
4,2013-01-05,1115,6.0,4771.0,339.0,1.0,0.0,0,1.0,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618468,2014-06-26,127,4.0,3335.0,480.0,1.0,0.0,0,0.0,d,a,1350.0,12.0,2005.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
618469,2014-06-27,127,5.0,4724.0,591.0,1.0,0.0,0,0.0,d,a,1350.0,12.0,2005.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
618470,2014-06-28,127,6.0,5398.0,716.0,1.0,0.0,0,0.0,d,a,1350.0,12.0,2005.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
618471,2014-06-29,127,7.0,0.0,0.0,0.0,0.0,0,0.0,d,a,1350.0,12.0,2005.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"


In [117]:
#Train test split processing

X = total.drop(columns='Sales')
y = total.loc[:,'Sales']

In [118]:
from sklearn.model_selection import train_test_split

In [119]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [120]:
todo = [y_train, X_train]
todo_test = [y_test, X_test]

In [121]:
#Merge train data (X and y) to process them together
df_train = pd.concat(todo, axis=1)
df_test = pd.concat(todo_test, axis=1)

In [122]:
#Profilibg to check data
from pandas_profiling import ProfileReport

In [123]:
#prof1 = ProfileReport(df_train)

In [124]:
#prof1.to_file(output_file='prof1.html')

In [125]:
#Drop na values in sales

df_train = df_train.dropna(subset =['Sales'])
df_test = df_test.dropna(subset =['Sales'])

#Drop columns Customers and PromoInterval

df_train = df_train.drop(columns=['Customers','PromoInterval'])
df_test = df_test.drop(columns=['Customers','PromoInterval'])

In [126]:
df_train['Sales'].isnull().value_counts()

False    480036
Name: Sales, dtype: int64

## Data Preparation

In [127]:
# Competitor distance : missing values

df_train['CompetitionDistance'] = df_train.loc[:,'CompetitionDistance'].fillna(df_train['CompetitionDistance'].median())
                                                                            
df_test['CompetitionDistance'] = df_test.loc[:,'CompetitionDistance'].fillna(df_test['CompetitionDistance'].median())

In [128]:
df_train.isnull().sum()

Sales                             0
Date                              0
Store                             0
DayOfWeek                     14287
Open                          14391
Promo                         14491
StateHoliday                  14465
SchoolHoliday                 14612
StoreType                         0
Assortment                        0
CompetitionDistance               0
CompetitionOpenSinceMonth    152707
CompetitionOpenSinceYear     152707
Promo2                            0
Promo2SinceWeek              235438
Promo2SinceYear              235438
dtype: int64

In [129]:
df_train['Open'].isna().sum()

14391

In [130]:
#Open: filling na using Sales

df_train.loc[df_train['Sales']> 0, 'Open']=1
df_train.loc[df_train['Sales']== 0, 'Open']=0

df_test.loc[df_test['Sales']> 0, 'Open']=1
df_test.loc[df_test['Sales']== 0, 'Open']=0

In [131]:
df_train['Open'].value_counts()

1.0    397880
0.0     82156
Name: Open, dtype: int64

In [132]:
#SatateHoloiday: fillna with 0 and get a string
df_train['StateHoliday'] = df_train.loc[:,'StateHoliday'].fillna('0')
df_train['StateHoliday'] = df_train['StateHoliday'].replace(0.0, '0')

df_test['StateHoliday'] = df_test.loc[:,'StateHoliday'].fillna('0')
df_test['StateHoliday'] = df_test['StateHoliday'].replace(0.0, '0')

In [133]:
df_train.StateHoliday.unique()

<StringArray>
['0', 'a', 'c', 'b']
Length: 4, dtype: string

In [134]:
df_train['SchoolHoliday'].isnull().value_counts()

False    465424
True      14612
Name: SchoolHoliday, dtype: int64

In [137]:
#Schoolholiday fillna using 0
df_train['SchoolHoliday'] = df_train['SchoolHoliday'].fillna(0)
df_test['SchoolHoliday'] = df_test['SchoolHoliday'].fillna(0)

In [138]:
#Tranform Data into datatime pd to get the day, week and year
df_train['Date'] = pd.to_datetime(df_train['Date'])
df_train['Year'] = pd.DatetimeIndex(df_train['Date']).year
df_train['Month'] = pd.DatetimeIndex(df_train['Date']).month
df_train['Day'] = pd.DatetimeIndex(df_train['Date']).day

df_test['Date'] = pd.to_datetime(df_test['Date'])
df_test['Year'] = pd.DatetimeIndex(df_test['Date']).year
df_test['Month'] = pd.DatetimeIndex(df_test['Date']).month
df_test['Day'] = pd.DatetimeIndex(df_test['Date']).day

In [139]:
#prof2 = ProfileReport(df_train)

In [140]:
#prof2.to_file(output_file='prof2.html')

In [141]:
df_train['Promo'].isna().value_counts()

False    465545
True      14491
Name: Promo, dtype: int64

In [142]:
#Promo missing values (0)
df_train['Promo'] = df_train.loc[:,'Promo'].fillna(0)
df_test['Promo'] = df_test.loc[:,'Promo'].fillna(0)

In [143]:
df_train['Promo2'].isna().value_counts()

False    480036
Name: Promo2, dtype: int64

In [144]:
from datetime import datetime 
from datetime import date 

## Create a new column that is a real date.
## For a time series we need real dates.
df_train['sales_date'] = pd.to_datetime(df_train['Date'])
total.info()

df_test['sales_date'] = pd.to_datetime(df_test['Date'])
total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 618473 entries, 0 to 618472
Data columns (total 18 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Date                       618473 non-null  object 
 1   Store                      618473 non-null  string 
 2   DayOfWeek                  600036 non-null  float64
 3   Sales                      600028 non-null  float64
 4   Customers                  599957 non-null  float64
 5   Open                       599832 non-null  float64
 6   Promo                      599791 non-null  float64
 7   StateHoliday               599873 non-null  string 
 8   SchoolHoliday              599686 non-null  float64
 9   StoreType                  618473 non-null  string 
 10  Assortment                 618473 non-null  string 
 11  CompetitionDistance        616838 non-null  float64
 12  CompetitionOpenSinceMonth  422043 non-null  float64
 13  CompetitionOpenSinceYear   42

In [145]:
def create_a_timeseries_dummy_from_weeks(df, var_week, var_year, date_new, var_dummy):
    #Create a date out of the Promo2 variables so that we can create a time-series variable for Promo2
    df['temp_date'] = df[var_year] * 1000 + df[var_week]  * 10 + 0
    df[date_new] = pd.to_datetime(df['temp_date'], format='%Y%W%w')
    ## Create an array to Group the data by group. and make dummy Store by Store
    for Store, grouped in df.groupby('Store'):
      if [df['sales_date'] >= df[date_new]]:
        df[var_dummy] = 1
      else:
        df[var_dummy] = 0
    df = df.drop(columns=['temp_date'])
    df.head()

    
def create_a_timeseries_dummy_from_months(df, var_month, var_year, date_new, var_dummy):
    #Create a date out of the Competition variables so that we can create a time-series variable for Competitioin
    df['temp_date'] = df[var_year] * 1000 + df[var_month]  * 10 + 0
    ## Create an array to Group the data by group. and make dummy Store by Store
    df[date_new] = pd.to_datetime(df['temp_date'], format='%Y%m%w')
    for Store, grouped in df.groupby('Store'):
      if [df['sales_date'] >= df[date_new]]:
        df[var_dummy] = 1
      else:
        df[var_dummy] = 0
    df = df.drop(columns=['temp_date'])    
    df.head()

In [146]:
create_a_timeseries_dummy_from_weeks(df=df_train        ,
                          var_week='Promo2SinceWeek'  ,
                          var_year='Promo2SinceYear'  ,
                          date_new='Promo2_start_date',
                          var_dummy='Promo2_yes'
                                    )

create_a_timeseries_dummy_from_weeks(df=df_test        ,
                          var_week='Promo2SinceWeek'  ,
                          var_year='Promo2SinceYear'  ,
                          date_new='Promo2_start_date',
                          var_dummy='Promo2_yes'
                                    )

In [147]:
create_a_timeseries_dummy_from_months(df=df_train                  ,
                          var_month='CompetitionOpenSinceMonth'  ,
                          var_year='CompetitionOpenSinceYear'   ,
                          date_new='Competition_start_date'    ,
                          var_dummy='Competition_yes'
                                    )

create_a_timeseries_dummy_from_months(df=df_test                 ,
                          var_month='CompetitionOpenSinceMonth'  ,
                          var_year='CompetitionOpenSinceYear'   ,
                          date_new='Competition_start_date'    ,
                          var_dummy='Competition_yes'
                                    )


In [148]:
df_train

,Sales,Date,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,...,Promo2SinceYear,Year,Month,Day,sales_date,temp_date,Promo2_start_date,Promo2_yes,Competition_start_date,Competition_yes
290482,5982.0,2013-11-19,175,2.0,1.0,1.0,0,0.0,c,a,...,NaN,2013,11,19,2013-11-19,NaN,NaT,1,NaT,1
139915,8256.0,2013-04-13,468,6.0,1.0,0.0,0,0.0,c,c,...,NaN,2013,4,13,2013-04-13,2012090.0,NaT,1,2012-09-01,1
318657,4714.0,2013-07-22,941,1.0,1.0,0.0,0,0.0,a,a,...,2013.0,2013,7,22,2013-07-22,2011120.0,2013-08-11,1,2011-12-01,1
361617,13744.0,2014-01-06,894,1.0,1.0,1.0,0,1.0,a,a,...,NaN,2014,1,6,2014-01-06,2012110.0,NaT,1,2012-11-01,1
374646,7308.0,2013-05-24,975,5.0,1.0,0.0,0,0.0,a,c,...,2011.0,2013,5,24,2013-05-24,NaN,2011-04-10,1,NaT,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110268,6523.0,2014-01-13,536,1.0,1.0,0.0,0,0.0,a,c,...,2013.0,2014,1,13,2014-01-13,2002090.0,2013-08-11,1,2002-09-01,1
259178,10356.0,2014-05-09,256,5.0,1.0,0.0,0,0.0,a,c,...,2014.0,2014,5,9,2014-05-09,2005090.0,2014-03-16,1,2005-09-01,1
365838,2763.0,2013-06-25,873,2.0,1.0,0.0,0,0.0,a,a,...,NaN,2013,6,25,2013-06-25,2008110.0,NaT,1,2008-11-01,1
131932,1581.0,2014-04-05,483,6.0,1.0,0.0,0,0.0,a,c,...,2010.0,2014,4,5,2014-04-05,2011090.0,2010-04-04,1,2011-09-01,1


In [149]:
#Drop columns for RandomForest Regressor
df_train = df_train.drop(columns=['Promo2_start_date','Competition_start_date','temp_date','Date', 'Promo2'])
df_train = df_train.drop(columns=['CompetitionOpenSinceYear','Promo2SinceWeek','Promo2SinceYear','CompetitionOpenSinceMonth'])

df_test = df_test.drop(columns=['Promo2_start_date','Competition_start_date','temp_date','Date', 'Promo2'])
df_test = df_test.drop(columns=['CompetitionOpenSinceYear','Promo2SinceWeek','Promo2SinceYear','CompetitionOpenSinceMonth'])

In [150]:
df_train

,Sales,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Year,Month,Day,sales_date,Promo2_yes,Competition_yes
290482,5982.0,175,2.0,1.0,1.0,0,0.0,c,a,4130.0,2013,11,19,2013-11-19,1,1
139915,8256.0,468,6.0,1.0,0.0,0,0.0,c,c,5260.0,2013,4,13,2013-04-13,1,1
318657,4714.0,941,1.0,1.0,0.0,0,0.0,a,a,1200.0,2013,7,22,2013-07-22,1,1
361617,13744.0,894,1.0,1.0,1.0,0,1.0,a,a,190.0,2014,1,6,2014-01-06,1,1
374646,7308.0,975,5.0,1.0,0.0,0,0.0,a,c,9630.0,2013,5,24,2013-05-24,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110268,6523.0,536,1.0,1.0,0.0,0,0.0,a,c,4700.0,2014,1,13,2014-01-13,1,1
259178,10356.0,256,5.0,1.0,0.0,0,0.0,a,c,80.0,2014,5,9,2014-05-09,1,1
365838,2763.0,873,2.0,1.0,0.0,0,0.0,a,a,2040.0,2013,6,25,2013-06-25,1,1
131932,1581.0,483,6.0,1.0,0.0,0,0.0,a,c,2310.0,2014,4,5,2014-04-05,1,1


In [152]:
df_train.DayOfWeek = df_train["DayOfWeek"].fillna(1)
df_test.DayOfWeek = df_test["DayOfWeek"].fillna(1)

### Encodings

In [154]:
import category_encoders as ce

In [155]:
# these are already dummies 'Open','Promo', 'SchoolHoliday',,'Promo2' 'DayOfWeek',
ce_one = ce.OneHotEncoder(cols=['StateHoliday','StoreType','Assortment'])  #instantiate the encoder

df_train = ce_one.fit_transform(df_train)
df_test = ce_one.transform(df_test)

In [156]:
df_train.columns

Index(['Sales', 'Store', 'DayOfWeek', 'Open', 'Promo', 'StateHoliday_1',
       'StateHoliday_2', 'StateHoliday_3', 'StateHoliday_4', 'SchoolHoliday',
       'StoreType_1', 'StoreType_2', 'StoreType_3', 'StoreType_4',
       'Assortment_1', 'Assortment_2', 'Assortment_3', 'CompetitionDistance',
       'Year', 'Month', 'Day', 'sales_date', 'Promo2_yes', 'Competition_yes'],
      dtype='object')

In [157]:
df_train.head()

,Sales,Store,DayOfWeek,Open,Promo,StateHoliday_1,StateHoliday_2,StateHoliday_3,StateHoliday_4,SchoolHoliday,...,Assortment_1,Assortment_2,Assortment_3,CompetitionDistance,Year,Month,Day,sales_date,Promo2_yes,Competition_yes
290482,5982.0,175,2.0,1.0,1.0,1,0,0,0,0.0,...,1,0,0,4130.0,2013,11,19,2013-11-19,1,1
139915,8256.0,468,6.0,1.0,0.0,1,0,0,0,0.0,...,0,1,0,5260.0,2013,4,13,2013-04-13,1,1
318657,4714.0,941,1.0,1.0,0.0,1,0,0,0,0.0,...,1,0,0,1200.0,2013,7,22,2013-07-22,1,1
361617,13744.0,894,1.0,1.0,1.0,1,0,0,0,1.0,...,1,0,0,190.0,2014,1,6,2014-01-06,1,1
374646,7308.0,975,5.0,1.0,0.0,1,0,0,0,0.0,...,0,1,0,9630.0,2013,5,24,2013-05-24,1,1


In [158]:
#use df_randomf for onehotencoding
df_randomf = df_train.copy()
df_randomf_test = df_test.copy()

In [159]:
df_randomf

,Sales,Store,DayOfWeek,Open,Promo,StateHoliday_1,StateHoliday_2,StateHoliday_3,StateHoliday_4,SchoolHoliday,...,Assortment_1,Assortment_2,Assortment_3,CompetitionDistance,Year,Month,Day,sales_date,Promo2_yes,Competition_yes
290482,5982.0,175,2.0,1.0,1.0,1,0,0,0,0.0,...,1,0,0,4130.0,2013,11,19,2013-11-19,1,1
139915,8256.0,468,6.0,1.0,0.0,1,0,0,0,0.0,...,0,1,0,5260.0,2013,4,13,2013-04-13,1,1
318657,4714.0,941,1.0,1.0,0.0,1,0,0,0,0.0,...,1,0,0,1200.0,2013,7,22,2013-07-22,1,1
361617,13744.0,894,1.0,1.0,1.0,1,0,0,0,1.0,...,1,0,0,190.0,2014,1,6,2014-01-06,1,1
374646,7308.0,975,5.0,1.0,0.0,1,0,0,0,0.0,...,0,1,0,9630.0,2013,5,24,2013-05-24,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110268,6523.0,536,1.0,1.0,0.0,1,0,0,0,0.0,...,0,1,0,4700.0,2014,1,13,2014-01-13,1,1
259178,10356.0,256,5.0,1.0,0.0,1,0,0,0,0.0,...,0,1,0,80.0,2014,5,9,2014-05-09,1,1
365838,2763.0,873,2.0,1.0,0.0,1,0,0,0,0.0,...,1,0,0,2040.0,2013,6,25,2013-06-25,1,1
131932,1581.0,483,6.0,1.0,0.0,1,0,0,0,0.0,...,0,1,0,2310.0,2014,4,5,2014-04-05,1,1


In [160]:
#ce_one = ce.OneHotEncoder(cols=['Year','Month'])
#df_randomf = ce_one.fit_transform(df_randomf)


#df_randomf_test = ce_one.fit_transform(df_randomf_test)

In [161]:
df_randomf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 480036 entries, 290482 to 121958
Data columns (total 24 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Sales                480036 non-null  float64       
 1   Store                480036 non-null  string        
 2   DayOfWeek            480036 non-null  float64       
 3   Open                 480036 non-null  float64       
 4   Promo                480036 non-null  float64       
 5   StateHoliday_1       480036 non-null  int64         
 6   StateHoliday_2       480036 non-null  int64         
 7   StateHoliday_3       480036 non-null  int64         
 8   StateHoliday_4       480036 non-null  int64         
 9   SchoolHoliday        480036 non-null  float64       
 10  StoreType_1          480036 non-null  int64         
 11  StoreType_2          480036 non-null  int64         
 12  StoreType_3          480036 non-null  int64         
 13  StoreType

In [162]:
#Drop Day and Sales_date 

df_randomf = df_randomf.drop(columns=['Day','sales_date'])

df_randomf_test = df_randomf_test.drop(columns=['Day','sales_date'])

In [163]:
#Remove for Sales 0 values (regarding metrics)
df_randomf_clean = df_randomf['Sales'].dropna(axis=0)
df_randomf_clean

df_randomf_clean_test = df_randomf_test['Sales'].dropna(axis=0)

df_randomf_clean1 = df_randomf.copy()
df_randomf_clean1.loc[:,'Sales'] = df_randomf['Sales'].dropna(axis=0)
df_randomf_clean1['Sales'].isnull().value_counts()
df_randomf_clean1 = df_randomf_clean1.drop(df_randomf_clean1[df_randomf_clean1['Sales']==0].index)


df_randomf_clean1_test = df_randomf_test.copy()
df_randomf_clean1_test.loc[:,'Sales'] = df_randomf_test['Sales'].dropna(axis=0)
df_randomf_clean1_test['Sales'].isnull().value_counts()
df_randomf_clean1_test = df_randomf_clean1_test.drop(df_randomf_clean1_test[df_randomf_clean1_test['Sales']==0].index)



df_randomf_clean1.isna().sum()

Sales                  0
Store                  0
DayOfWeek              0
Open                   0
Promo                  0
StateHoliday_1         0
StateHoliday_2         0
StateHoliday_3         0
StateHoliday_4         0
SchoolHoliday          0
StoreType_1            0
StoreType_2            0
StoreType_3            0
StoreType_4            0
Assortment_1           0
Assortment_2           0
Assortment_3           0
CompetitionDistance    0
Year                   0
Month                  0
Promo2_yes             0
Competition_yes        0
dtype: int64

In [164]:
#Split data
X_train = df_randomf_clean1.drop(columns=['Sales'])
y_train = np.asarray(df_randomf_clean1.loc[:,'Sales'])

X_test = df_randomf_clean1_test.drop(columns=['Sales'])
y_test = np.asarray(df_randomf_clean1_test.loc[:,'Sales'])

In [165]:
# Scale 
scal = StandardScaler()
X_train = scal.fit_transform(X_train)

X_test = scal.fit_transform(X_test)

## Model Development

#### Linear Regression

In [167]:
#Model Linear Regression
from sklearn.linear_model import LinearRegression

regr = LinearRegression()
regr.fit(X_train, y_train)

LinearRegression()

In [168]:
#Prediction on training Data

y_pred_train = regr.predict(X_train)

y_pred_test = regr.predict(X_test)

In [169]:
#Building required metric
def metric(preds, actuals):
    preds = preds.reshape(-1)
    actuals = actuals.reshape(-1)
    assert preds.shape == actuals.shape
    return 100 * np.linalg.norm((actuals - preds) / actuals) / np.sqrt(preds.shape[0])

In [170]:
train_s = metric(y_pred_train,y_train)

test_s = metric(y_pred_test,y_test)

train_s

52.3873876303537

In [171]:
test_s

676482166397.538

#### Gradient Boosted Trees

In [172]:
#import libraries
import xgboost as xgb

In [ ]:
# instanstiate the regressor 
xgbr = xgb.XGBRegressor(n_estimators = 1500, max_depth = 7, learning_rate = 0.15, n_jobs= -1)

#fit the regressor
xgbr.fit(X_train, y_train)

In [ ]:
#predict with xgbr
y_pred_xgbr = xgbr.predict(X_test)

y_pred_xgbr_train = xgbr.predict(X_train)

In [ ]:
metric_xgbr_train = metric(y_pred_xgbr_train, y_train)

metric_xgbr_train

In [ ]:
#Accuracy
metric_xgbr = metric(y_pred_xgbr,y_test)

metric_xgbr